# Module Imports

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, Dense, GlobalAveragePooling1D
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

random.seed(42)
np.random.seed(42)

# Load and Process Data

In [3]:
# Load MovieLens dataset
data_path = 'ml-100k/u.data'
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_csv(data_path, sep='\t', names=column_names)

In [18]:
movies = pd.read_csv('ml-100k/movies.csv', sep=',')

In [30]:
# Preprocess the dataset
user_ids = data['user_id'].unique()
item_ids = data['item_id'].unique()
uid2idx = {uid: idx for idx, uid in enumerate(user_ids)}
iid2idx = {iid: idx for idx, iid in enumerate(item_ids)}
idx2uid = {idx: uid for uid, idx in uid2idx.items()}
idx2iid = {idx: iid for iid, idx in iid2idx.items()}
data['user_id'] = data['user_id'].map(uid2idx)
data['item_id'] = data['item_id'].map(iid2idx)
movies['item_id'] = movies['movieId'].map(iid2idx)

In [49]:
# Sort data by timestamp
train_data = data.sort_values(by='timestamp')

In [7]:
def generate_sequences_and_labels(data, user_ids, item_ids, seq_length, num_items):
    sequences, labels = [], []

    for user_id in user_ids:
        user_data = data[data['user_id'] == user_id]
        user_items = user_data['item_id'].values.tolist()
        
        # Create sequences of fixed length for each user
        for i in range(len(user_items) - seq_length):
            sequences.append(user_items[i:i + seq_length])
            
            label = np.zeros(num_items)
            if user_items[i + seq_length] in user_items[i:i + seq_length]:
                label[user_items[i + seq_length]] = 1  # Positive feedback
            else:
                label[user_items[i + seq_length]] = 0  # Negative feedback
            labels.append(label)

    sequences = np.array(sequences)
    labels = np.array(labels)
    return sequences, labels

In [8]:
seq_length = 5  # Adjust this value based on your requirements
train_sequences, train_labels = generate_sequences_and_labels(train_data, user_ids, item_ids, seq_length, len(item_ids))

# Visualize Data

In [9]:
data.head()

,user_id,item_id,rating,timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596


In [10]:
print('Number of users: ', len(user_ids))
print('Number of items: ', len(item_ids))
print('Number of interactions: ', len(data))
print('Average rating: ', data['rating'].mean())
print('Start Date: ', pd.to_datetime(data['timestamp'].min(), unit='s'))
print('End Date: ', pd.to_datetime(data['timestamp'].max(), unit='s'))

Number of users:  943
Number of items:  1682
Number of interactions:  100000
Average rating:  3.52986
Start Date:  1997-09-20 03:05:10
End Date:  1998-04-22 23:10:38


# NextItNet Model

In [11]:
def create_nextitnet_model(num_items, embedding_dim, num_layers, kernel_size, dropout_rate):
    input_layer = Input(shape=(None,))
    x = Embedding(num_items, embedding_dim)(input_layer)
    
    for _ in range(num_layers):
        x = Conv1D(embedding_dim, kernel_size, padding='causal', activation='relu')(x)
        x = Dropout(dropout_rate)(x, training=True)  # Enable dropout during inference for uncertainty estimation

    x = GlobalAveragePooling1D()(x)
    output_layer = Dense(num_items, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

In [12]:
num_items = len(item_ids)
embedding_dim = 64
num_layers = 3
kernel_size = 3
dropout_rate = 0.5

nextitnet = create_nextitnet_model(num_items, embedding_dim, num_layers, kernel_size, dropout_rate)
nextitnet.compile(optimizer='adam', loss='binary_crossentropy')

In [13]:
history = nextitnet.fit(train_sequences, train_labels, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
471/471 [==============================] - 27s 8ms/step - loss: 0.0405 - val_loss: 6.4814e-07
Epoch 2/10
471/471 [==============================] - 3s 7ms/step - loss: 3.8388e-07 - val_loss: 4.7455e-07
Epoch 3/10
471/471 [==============================] - 3s 6ms/step - loss: 2.9732e-07 - val_loss: 4.3149e-07
Epoch 4/10
471/471 [==============================] - 3s 6ms/step - loss: 2.1637e-07 - val_loss: 3.5134e-07
Epoch 5/10
471/471 [==============================] - 3s 6ms/step - loss: 1.7083e-07 - val_loss: 3.5041e-07
Epoch 6/10
471/471 [==============================] - 3s 7ms/step - loss: 1.3873e-07 - val_loss: 1.9137e-07
Epoch 7/10
471/471 [==============================] - 3s 6ms/step - loss: 1.1220e-07 - val_loss: 2.0596e-07
Epoch 8/10
471/471 [==============================] - 3s 7ms/step - loss: 9.1439e-08 - val_loss: 1.8184e-07
Epoch 9/10
471/471 [==============================] - 3s 7ms/step - loss: 7.3155e-08 - val_loss: 1.6583e-07
Epoch 10/10
471/471 [==========

# Uncertainty Score

In [14]:
def estimate_uncertainty(model, user_sequence, num_samples=100):
    user_sequence = np.expand_dims(user_sequence, axis=0)
    predictions = np.stack([model.predict(user_sequence) for _ in range(num_samples)], axis=-1)
    means = np.mean(predictions, axis=-1)
    variances = np.var(predictions, axis=-1)
    return means, variances

def get_movie_titles(recommendations, movies_df):
    movie_titles = []
    for index in recommendations:
        movie_title = movies_df.loc[index, 'title']
        movie_titles.append(movie_title)
    return movie_titles

In [47]:
# Grab X random item ids
test_sequence_length = 3
user_sequence = list(np.random.choice(item_ids, test_sequence_length))  # A sample user sequence
means, variances = estimate_uncertainty(nextitnet, user_sequence)

k = 5
top_k_recommendations = np.argsort(means[0])[-k:][::-1]  # Get top 5 recommended items
top_k_uncertainties = variances[0][top_k_recommendations]  # Get corresponding uncertainty scores

1/1 [==============================] - 0s 17ms/step


In [48]:
top_k_movie_titles = get_movie_titles(top_k_recommendations, movies)
for i in range(len(top_k_movie_titles)):
    print(top_k_movie_titles[i], 'Uncertainty Score: ', top_k_uncertainties[i])

Bread and Chocolate (Pane e cioccolata) (1973) Uncertainty Score:  2.1802765e-07
Batman & Robin (1997) Uncertainty Score:  2.0597514e-07
Murder, My Sweet (1944) Uncertainty Score:  1.619843e-07
Love and Other Catastrophes (1996) Uncertainty Score:  2.0653522e-07
Amateur (1994) Uncertainty Score:  1.5016994e-07
